# LLM Factory 튜토리얼

이 튜토리얼에서는 다양한 LLM(Large Language Model) 제공업체의 모델들을 통합적으로 관리하고 사용하는 방법을 배워보겠습니다.

## 주요 기능
- OpenAI, Anthropic, Naver 등 다양한 LLM 제공업체 지원
- 팩토리 패턴을 통한 일관된 모델 생성 인터페이스
- 중앙 집중식 모델 설정 관리
- 커스텀 모델 설정 지원

## 1. 환경 설정

먼저 필요한 환경변수를 설정합니다. 각 LLM 제공업체의 API 키가 필요합니다.

In [1]:
from dotenv import load_dotenv

In [2]:
load_dotenv(override=True)

True

In [3]:
import os

# API 키 설정
# os.environ["OPENAI_API_KEY"] = "your-openai-api-key"
# os.environ["ANTHROPIC_API_KEY"] = "your-anthropic-api-key"
# os.environ["NCP_APIGW_API_KEY"] = "your-naver-apigw-key"
# os.environ["NCP_CLOVASTUDIO_API_KEY"] = "your-naver-studio-key"
# os.environ["DEEPSEEK_API_KEY"] = "your-deepseek-api-key"

## 2. 기본 사용법

### 2.1 사용 가능한 모델 확인

In [4]:
from langmetrics.llmfactory import LLMFactory

# 사용 가능한 모델 목록 확인
available_models = LLMFactory.get_model_list()
print("Available models:", available_models)

/Users/jin/anaconda3/envs/translate/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Available models: ['gpt-4o', 'gpt-4o-mini', 'deepseek-v3', 'deepseek-reasoner', 'claude-3.5-sonnet', 'claude-3.5-haiku', 'naver']


### 2.2 기본 모델 생성 및 사용

In [5]:
# GPT-4 모델 생성
gpt4_llm = LLMFactory.create_llm("gpt-4o-mini", temperature=0.7)

# Claude 모델 생성
# claude_llm = LLMFactory.create_llm("claude-3.5-sonnet", temperature=0.5)

# 네이버 모델 생성
naver_llm = LLMFactory.create_llm("naver", temperature=0.8)

# deepseek 모델 생성
deepseek_llm = LLMFactory.create_llm('deepseek-v3', temperature=0.1)

## 3. 커스텀 모델 설정 사용

get_config 메소드를 활용하면 현재 Model의 Config 값을 확인할 수 있습니다.

In [6]:
print(LLMFactory.get_config('deepseek-v3'))

ModelConfig(model_name='deepseek-chat', api_key='sk-73508341cbe64cc3916f480fd37e91f2', api_base='https://api.deepseek.com', seed=66, max_tokens=8000, provider='deepseek')


기본 제공되는 모델 외에도 커스텀 설정으로 새로운 모델을 생성할 수 있습니다.

In [12]:
from langmetrics.config import ModelConfig

# 커스텀 모델 설정 생성
custom_config = ModelConfig(
    model_name="gpt-4-turbo-preview",
    api_base="https://api.openai.com/v1",
    api_key=os.getenv("OPENAI_API_KEY"),
    max_tokens=128000,
    seed=66,
    provider="openai"
)

# 커스텀 설정으로 모델 생성
custom_llm = LLMFactory.create_llm(custom_config, temperature=0.7)

## 4. 로컬 모델 설정 사용

CustomModelConfig를 사용하여 LocalGPU를 사용하여 모델 사용하기

In [ ]:
from langmetrics.config import LocalModelConfig

local_config = LocalModelConfig(
    model_name="Qwen/Qwen2.5-7B-Instruct-1M",  # 사용할 모델 이름 (Qwen2.5-7B-Instruct-1M)
    port=30000,  # 서버가 실행될 포트 번호 (30000번 포트에서 대기)
    max_tokens=4000,  # input 토큰 수 + output 토큰 수
    gpus="1,2",  # 사용할 GPU ID (GPU 1번과 2번 사용)
    dp=1,  # 데이터 병렬 처리 (Data Parallelism) 활성화 (2개 GPU 사용), token output speed 증가
    tp=2   # 텐서 병렬 처리 (Tensor Parallelism) 활성화 (2개 GPU 사용), GPU VRAM 크기가 모자르다면 더 키워야함.
)

In [ ]:
# localllm은 서버를 local에서 실행시키기 때문에 부팅되는 시간이 존재합니다.
# local_llm = LLMFactory.create_llm(local_config, temperature=0.7)

# # 또한 서버는 terminate을 이용하여 끌 수 있습니다.
# local_llm.terminate()

## 4. 모델 사용 예시

생성된 모델을 사용하여 텍스트를 생성하는 예시입니다.

In [5]:
from langchain.schema import HumanMessage

# 텍스트 생성 예시
messages = [
    HumanMessage(content="만성콩팥병에 대해 설명해주세요.")
]

# 각 모델로 응답 생성
gpt4_response = gpt4_llm.invoke(messages)
deepseek_response = deepseek_llm.invoke(messages)
naver_response = naver_llm.invoke(messages)

print("=== GPT-4 Response ===")
print(gpt4_response.content)
print("\n=== DeepSeek Response ===")
print(deepseek_response.content)
print("\n=== Naver Response ===")
print(naver_response.content)


=== GPT-4 Response ===
만성콩팥병(Chronic Kidney Disease, CKD)은 콩팥(신장)의 기능이 점진적으로 저하되는 질환을 말합니다. 이 병은 여러 가지 원인에 의해 발생할 수 있으며, 특히 당뇨병, 고혈압, 그리고 심혈관 질환과 같은 만성 질환이 주요 원인입니다. 만성콩팥병은 일반적으로 3개월 이상 지속되며, 신장의 기능이 악화됨에 따라 여러 가지 합병증이 발생할 수 있습니다.

### 만성콩팥병의 단계
만성콩팥병은 신장 기능의 정도에 따라 5단계로 나눌 수 있습니다:
1. **1단계**: 신장 기능이 정상이나, 신장 손상이 존재하는 경우.
2. **2단계**: 경미한 신장 기능 저하 (GFR 60-89 mL/min).
3. **3단계**: 중간 정도의 신장 기능 저하 (GFR 30-59 mL/min). 이 단계는 다시 3a(45-59)와 3b(30-44)로 나눌 수 있습니다.
4. **4단계**: 심각한 신장 기능 저하 (GFR 15-29 mL/min).
5. **5단계**: 신부전 (GFR < 15 mL/min)으로, 이 단계에서는 투석이나 신장 이식이 필요할 수 있습니다.

### 증상
만성콩팥병은 초기에는 증상이 거의 없거나 경미할 수 있습니다. 하지만 병이 진행됨에 따라 다음과 같은 증상이 나타날 수 있습니다:
- 피로감과 무기력
- 부종 (특히 발과 발목)
- 고혈압
- 소변의 변화 (양, 색, 냄새)
- 식욕 감소와 체중 감소
- 메스꺼움 및 구토
- 가려움증

### 진단
만성콩팥병은 혈액 검사, 소변 검사, 그리고 영상 검사 등을 통해 진단됩니다. 특히 혈액에서 크레아티닌 수치를 측정하여 신장 기능을 평가하며, GFR(사구체 여과율)을 계산하여 병의 단계를 정합니다.

### 치료
만성콩팥병의 치료는 주로 원인 질환의 관리와 신장 기능 저하의 진행을 늦추는 데 중점을 둡니다. 일반적인 치료법은 다음과 같습니다:
- 혈압 조절 (항고혈압제 사용)
- 당뇨 관리
- 식이요법 (단백질, 나트륨, 칼륨 섭취 조절)
- 

## 정리

이 튜토리얼에서는 LLMFactory를 사용하여:
1. 다양한 LLM 제공업체의 모델을 생성하고
2. 커스텀 모델 설정을 만들어 사용하며
3. 생성된 모델로 텍스트를 생성하고
4. 에러 상황을 처리하는 방법을 배웠습니다.

LLMFactory는 여러 LLM 제공업체의 모델을 일관된 인터페이스로 사용할 수 있게 해주며,
중앙 집중식 설정 관리를 통해 모델 설정을 효율적으로 관리할 수 있습니다.